In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Estimation

### Sample Median

In [ ]:
sf = Table.read_table('san_francisco_2015.csv')
sf.show(5)

**Question:** What is this dataset?

In [ ]:
salary_sf.group(0).barh(0)

In [ ]:
# skip

In [ ]:
# skip

In [ ]:
# skip

In [ ]:
# We only care about salary for now
salary_sf = sf.select(3, 11, 21)
salary_sf

**Question:** Who is making the most money?

**Question:** Who is making the least money?

**Question:** What is the typical salary? Should we compute the mean or median for this?

Lets compute both and then compare, we will look at the histogram of total comps at the end

How can we compute the median with what we covered today?
<details>
<summary>Solution</summary>
  percentile(50, sf.column('Total Compensation'))
</details>  

In [ ]:
pop_median = ...
pop_median

In [ ]:
sf_bins = np.arange(0, 700000, 25000)
sf.hist('Total Compensation', bins=sf_bins)
plots.title('Population Distribution');

### Estimating Salaries
Say we dont have access to all the salaries. What would we do to compute the 50% percentile of salaries?

In [ ]:
# skip

In [ ]:
# skip

In [ ]:
# skip

In [ ]:
our_sample = sf.sample(300, with_replacement=False)
our_sample.show(5)

In [ ]:
estimate_median = percentile(50, our_sample.column('Total Compensation'))
estimate_median

In [ ]:
our_sample.hist('Total Compensation', bins=sf_bins)
plots.title('Sample Distribution');

**Question:** How far off is our estimate from the true median?

In [ ]:
pop_median, estimate_median, abs(pop_median - estimate_median)

## Variability of the Estimate

Let's implement the following function:

<details>
<summary>Solution</summary>
  our_sample = sf.sample(samp_size, with_replacement=False)
  return percentile(50, our_sample.column('Total Compensation'))
</details>


In [ ]:
def generate_sample_median(samp_size):
    """
    Given a sample size
    Returns the median of a random sample of samp_size from the sf table"""
    # Question: sample with or without replacement?
    
    

sample_median = generate_sample_median(300)
sample_median

**Question:** What is our error?


<details>
<summary>Solution</summary>
  error = sample_median - pop_median
</details>


In [ ]:
error = ...
error

(back to slides)
# Quantifying Uncertainty

Lets take 1k estimates where each sample has 300 individuals


In [ ]:
sample_medians = make_array()

for i in np.arange(1000):
    new_median = generate_sample_median(300)
    sample_medians = np.append(sample_medians, new_median)
sample_medians

Let's plot the medians

In [ ]:
med_bins = np.arange(90000, 125001, 2500)
Table().with_column(
    'Sample Medians', sample_medians
).hist(bins = med_bins)

plots.scatter(pop_median, -1e-6, color="red");

Lets plot the errors

In [ ]:
err_bins = np.arange(-15000, 12501, 2500)
Table().with_column(
    'Errors', sample_medians - pop_median
).hist(bins = err_bins)

plots.scatter(0, -1e-6, color="red");

(back to slides)
# Bootstrap

In [ ]:
our_sample

In [ ]:
# Take a bootstrap (re)sample of size 300, WITH replacement

# Sample from our sample
boot_sample = our_sample.sample(with_replacement=True)
#boot_sample

Let's compare the median from our sample with the median of the boostrapped sample

In [ ]:
our_sample_median = percentile(50, our_sample.column('Total Compensation'))
boot_sample_median = percentile(50, boot_sample.column('Total Compensation'))

In [ ]:
# Show the bootstrap sample 
boot_sample.hist('Total Compensation', bins=sf_bins)
plots.title('1 Bootstrap sample');

print("Population Median =       ", pop_median)
print("Our Sample Median =       ", our_sample_median)
print("Bootstrap Sample Median = ", 
      percentile(50,boot_sample.column('Total Compensation')))

### Multiple Bootstraps
Let's take 1k bootstraps. Lets start by implementing the following function

<details>
<summary>Solution</summary>
   single_sample = our_sample.sample()
   return percentile(50, single_sample.column('Total Compensation'))
</details>  

In [ ]:
def one_bootstrap_median():
    single_sample = ...
    return ...

Now let's keep track of 1k bootstrapped median

In [ ]:
# Bootstrap our sample 1000 times
bootstrap_medians = ...
for i in np.arange(1000):
    new_median = ...
    bootstrap_medians = ...

Let's visualize these bootstrapped medians

In [ ]:
Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.scatter(pop_median, 0, color="red");
plots.scatter(our_sample_median, 0, color="blue");
plots.title('Bootstrap Medians (1K Bootstraps from our Sample)');

# 95% Confidence Interval

**Question**: How could we make an interval based on the middle 95% of bootstrap samples?

- *Hint 1:* Remember we stored the bootstrapped medians in an array called `bootstrap_medians`
- *Hint 2:* What did we learn about in the begining of this lecture?


<details>
<summary>Solution</summary>
  left = percentile(2.5, bootstrap_medians)
right = percentile(97.5, bootstrap_medians)
</details>  

In [ ]:
# Make an interval based on the middle 95% of bootstrap samples

left = ...
right = ...

In [ ]:
Table().with_column(
    'Bootstrap Medians', bootstrap_medians
).hist('Bootstrap Medians', bins=med_bins)

plots.plot([left, right], [0,0], color="gold",lw=3, zorder=1);
plots.scatter(pop_median, 0, color="red", zorder=2);
plots.scatter(our_sample_median, 0, color="blue", zorder=2);
plots.title('Bootstrap Medians (1K Bootstraps from our Sample)');

## Another Example: Mean Maternal Age

In [ ]:
# This time we have a sample, but no population data!
births = Table.read_table('baby.csv')
births.show(5)

How can we see a distribution of maternal ages?

<details>
<summary>Solution</summary>
  births.hist('Maternal Age')
</details>  

What is the mean age?

In [ ]:
mean_age = ...
mean_age

Now let's use bootstraping to find samples means

<details>
<summary>Solution</summary>
  np.mean(births.sample().column('Maternal Age'))
</details>  

In [ ]:
def one_bootstrap_mean():
    return ...

Let's compute 1k bootstrapped samples

In [ ]:
bootstrap_means = make_array()

for i in np.arange(1000):
    new_mean = one_bootstrap_mean()
    bootstrap_means = np.append(bootstrap_means, new_mean)
    
left = percentile(2.5, bootstrap_means)
right = percentile(97.5, bootstrap_means)

In [ ]:
Table().with_column('Bootstrap means', bootstrap_means).hist()

plots.plot([left,right], [0,0], color="gold", lw=3, zorder=1);
plots.scatter(mean_age,0,color="blue", zorder=2);
plots.title('Bootstrap Means (1K Bootstraps from our Sample)');

## Is this technique reliable?


Repeat this process 100 times and keep track of how many times the true population parameter was indeed in this interval

*Run this before discussing it because it will take about a minute to run*

In [ ]:
%%time
# This will take a while to run
intervals = Table().with_columns('Lower', make_array(), 'Upper', make_array())

for i in np.arange(100):
    sample_from_pop = births.sample(300, with_replacement=False)
    means = make_array()
    
    for j in np.arange(1000):
        resample = sample_from_pop.sample(with_replacement=True)
        mean = np.average(resample.column('Maternal Age'))
        means = np.append(means, mean)
        
    interval = make_array(
                percentile(2.5, means), 
                percentile(97.5, means))
    
    intervals.append(interval)

Now let's check how many times our intervals included the true parameter

In [ ]:
true_mean = np.average(births.column('Maternal Age'))
intervals.where('Lower', are.not_above(true_mean)).where('Upper', are.not_below(true_mean)).num_rows

In [ ]:
true_mean, intervals

**Question:** How many times was the true mean below our lower bound?

<details>
<summary>Solution</summary>
    intervals.where('Lower', are.above(true_mean))
</details>   

**Question:** How many times was the true mean above our upper bound?

<details>
<summary>Solution</summary>
    intervals.where('Upper', are.below(true_mean))
</details>   